In [9]:
from IPython.core.display import HTML
def css_styling():   
    styles = open("styles/custom.css", "r").read() 
    return HTML(styles) 

In [10]:
import pymongo
try:
    #create a credentials.txt file in this folder:
    #first line: Database username (not account username)
    #second line: Database username password (not account password)
    #third line: Databse url with port (you can find it in the cloud dashboard)
    #fourth lin: Database name
    if 'conn' in globals():
        conn.close()
        print("Closing")
    
    with open("credentials.txt", 'r') as f:
        [name,password,url,dbname]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb+srv://{}:{}@{}".format(name,password,url))
    print ("Connected successfully!!!")
    
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
conn
db = conn[dbname]
db


Closing
Connected successfully!!!


Database(MongoClient(host=['movielens-shard-00-01.8j2mw.mongodb.net:27017', 'movielens-shard-00-02.8j2mw.mongodb.net:27017', 'movielens-shard-00-00.8j2mw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='MovieLens', authsource='admin', replicaset='atlas-88psll-shard-0', tls=True), 'MovieLens')

In [11]:
import pandas as pd
file_paths = {
    "movies": "../data1/movies_clean.csv",
    "ratings": "../data1/ratings_clean.csv"
}

# Load the data into pandas dataframes
movies_data = pd.read_csv('../data1/movies_clean.csv', encoding='utf-8', sep = ',', header = 0)
ratings_data = pd.read_csv('../data1/ratings_clean.csv', encoding='ascii', sep=',', header = 0)

In [12]:
# Add column names
# movies_data = movies_df.to_dict("records")
# users_data = users_df.to_dict("records")
# ratings_data = ratings_df.to_dict("records")

In [13]:
print(movies_data[:5]) 
print(ratings_data[:5])  


   movieId                      title                            genres  year
0   210479                        NaN             ['Drama', 'Thriller']  2019
1    51372  "Great Performances" Cats                       ['Musical']  1998
2   281280                      "Sr."                   ['Documentary']  2022
3   136604        #1 Cheerleader Camp               ['Comedy', 'Drama']  2010
4   221850                     #Alive  ['Action', 'Horror', 'Thriller']  2020
   userId  movieId  rating               tstamp
0   60834     1608     2.0  1997-09-17 17:53:58
1   60834     1479     2.0  1997-09-17 17:53:58
2   60834     1588     1.0  1997-09-17 17:55:19
3   60834     1407     2.0  1997-09-17 17:56:19
4   60834     1422     2.0  1997-09-17 17:57:33


In [14]:
#MongoDB doesn't allow keys to be anything other than strings (no integers!)
def convert_keys_to_strings(data):
    return [{str(key): value for key, value in item.items()} for item in data.to_dict(orient='records')]

movies_data = convert_keys_to_strings(movies_data)
ratings_data = convert_keys_to_strings(ratings_data)

In [15]:
db.movies.insert_many(movies_data)

db.ratings.insert_many(ratings_data)
print("Datos insertados en MongoDB correctamente!")

Datos insertados en MongoDB correctamente!


In [16]:
for movie in db.movies.find().limit(5):
    print(movie)

for rating in db.ratings.find().limit(5):
    print(rating)

{'_id': ObjectId('673dc6e96f761f207f13933d'), 'movieId': 210479, 'title': nan, 'genres': "['Drama', 'Thriller']", 'year': '2019'}
{'_id': ObjectId('673dc6e96f761f207f13933e'), 'movieId': 51372, 'title': '"Great Performances" Cats', 'genres': "['Musical']", 'year': '1998'}
{'_id': ObjectId('673dc6e96f761f207f13933f'), 'movieId': 281280, 'title': '"Sr."', 'genres': "['Documentary']", 'year': '2022'}
{'_id': ObjectId('673dc6e96f761f207f139340'), 'movieId': 136604, 'title': '#1 Cheerleader Camp', 'genres': "['Comedy', 'Drama']", 'year': '2010'}
{'_id': ObjectId('673dc6e96f761f207f139341'), 'movieId': 221850, 'title': '#Alive', 'genres': "['Action', 'Horror', 'Thriller']", 'year': '2020'}
{'_id': ObjectId('673dc6fa6f761f207f1468b0'), 'userId': 60834, 'movieId': 1608, 'rating': 2.0, 'tstamp': '1997-09-17 17:53:58'}
{'_id': ObjectId('673dc6fa6f761f207f1468b1'), 'userId': 60834, 'movieId': 1479, 'rating': 2.0, 'tstamp': '1997-09-17 17:53:58'}
{'_id': ObjectId('673dc6fa6f761f207f1468b2'), 'user

In [17]:
conn.close()